In [1]:
"""
多输入模型：问答系统（Question Answering）

本示例构建一个基于双输入的问答模型：
- 输入1：参考文本（context）
- 输入2：问题（question）
- 输出：答案的类别（从候选答案库中选择）

模型架构：
1. 对文本和问题分别进行embedding
2. 使用LSTM提取序列特征
3. 拼接两路特征
4. 通过全连接层生成答案分类

应用场景：
- 阅读理解：根据文章回答问题
- 知识问答：从知识库检索答案
- 对话系统：理解上下文并生成响应
"""

from keras import layers
from keras.layers import Input
from keras.models import Model
import numpy as np
from keras.utils import to_categorical

# 设置随机种子保证可复现性
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 词汇表大小
text_vocabulary_size = 10000      # 参考文本词汇量
question_vocabulary_size = 10000  # 问题词汇量
answer_vocabulary_size = 500      # 候选答案数量

# 文本输入分支
text_input = Input(shape=(None,), dtype="int32", name="text")
embedded_text = layers.Embedding(
    text_vocabulary_size, 64)(text_input)
# LSTM编码文本序列为固定长度向量
encoded_text = layers.LSTM(32)(embedded_text)

# 问题输入分支
question_input = Input(shape=(None,), dtype="int32", name="question")
embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input)
# LSTM编码问题序列
encoded_question = layers.LSTM(16)(embedded_question)

# 特征融合：拼接文本和问题的编码
concatenated = layers.concatenate(
    [encoded_text, encoded_question], axis=-1)

# 答案生成：分类到候选答案空间
answer = layers.Dense(
    answer_vocabulary_size, 
    activation="softmax")(concatenated)

# 构建模型
model = Model([text_input, question_input], answer)
model.compile(optimizer="rmsprop", 
              loss="categorical_crossentropy", 
              metrics=["acc"])

print("问答模型架构：")
model.summary()

# 生成模拟数据进行验证
num_samples = 1000
max_length = 100

# 文本序列（词索引）
text = np.random.randint(1, text_vocabulary_size, 
                         size=(num_samples, max_length))
# 问题序列（词索引）
question = np.random.randint(1, question_vocabulary_size, 
                             size=(num_samples, max_length))
# 答案标签（one-hot编码）
answers = np.random.randint(answer_vocabulary_size, size=(num_samples,))
answer = to_categorical(answers, answer_vocabulary_size)

print("\n开始训练...")
history = model.fit([text, question], answer, 
                    epochs=3, 
                    batch_size=128,
                    validation_split=0.2)

print("\n模型训练完成")

2025-12-12 22:21:23.391779: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-12 22:21:23.398097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-12 22:21:23.406008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-12 22:21:23.408370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-12 22:21:23.414287: I tensorflow/core/platform/cpu_feature_guar

I0000 00:00:1765549284.457699   36384 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549284.479642   36384 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549284.480625   36384 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549284.482525   36384 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

问答模型架构：


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text (InputLayer)   │ (None, None)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 64)  │    640,000 │ text[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 32)  │    320,000 │ question[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32)        │     12,416 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 16)        │      3,136 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 48)        │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 500)       │     24,500 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

...

Epoch 3/3


1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - acc: 0.0156 - loss: 6.2071

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.0110 - loss: 6.2069 

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.0075 - loss: 6.2064 - val_acc: 0.0000e+00 - val_loss: 6.2129



模型训练完成
